In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tifffile
import time
from datetime import datetime
import pandas as pd
pd.set_option('display.max_colwidth', 250)
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import shape
import glob
from shapely.geometry import box
import rasterio
from rasterio.windows import Window
import json
from matplotlib.colors import LinearSegmentedColormap
red_to_green = LinearSegmentedColormap.from_list("RedGreen", ["red", "green"])

from src.train import train
from src.hyperparams import HyperParams, open_from_yaml
from src.data import *
from src.visualize import visualize_s2_concat_bands_path, visualize_s1_path, scale_img, visualize_ls_concat_bands_path, visualize_cb_mux_path, visualize_cb_wpm_path

In [ ]:
alerts_with_data = len(set([k.split("/")[1] for k in glob.glob(f"data/*/*/*.tif")]))
print(f"There is any satellite data for {alerts_with_data} alerts")

alerts_with_s1 = len(set([k.split("/")[1] for k in glob.glob(f"data/*/sentinel1/*.tif")]))
alerts_with_ls = len(set([k.split("/")[1] for k in glob.glob(f"data/*/landsat/*.tif")]))
alerts_with_mux = len(set([k.split("/")[1] for k in glob.glob(f"data/*/cbers4a/*MUX*.tif")]))
alerts_with_wpm = len(set([k.split("/")[1] for k in glob.glob(f"data/*/cbers4a/*WPM*.tif")]))
alerts_with_s2_l2a = len(set([k.split("/")[1] for k in glob.glob(f"data/*/sentinel2/*L2A*.tif")]))
alerts_with_s2_16d = len(set([k.split("/")[1] for k in glob.glob(f"data/*/sentinel2/*16D*.tif")]))
alerts_with_s2 = len(set([k.split("/")[1] for k in glob.glob(f"data/*/sentinel2/*L2A*.tif")] + [k.split("/")[1] for k in glob.glob(f"data/*/sentinel2/*16D*.tif")]))

print(f"There is any S2 data for {alerts_with_s2} alerts")
print(f"There is S2 L2A data for {alerts_with_s2_l2a} alerts")
print(f"There is S2 16D data for {alerts_with_s2_16d} alerts")
print(f"There is LS data for     {alerts_with_ls} alerts")
print(f"There is S1 data for     {alerts_with_s1} alerts")
print(f"There is MUX data for    {alerts_with_mux} alerts")
print(f"There is WPM data for    {alerts_with_wpm} alerts")

# S2 single sensor, single before/after image
- Now that I have managed to align L2A and 16D products for Sentinel2, I can train a model on their combined data.
- For 100 alerts, we have some data for 82 of them. From those 82, we have S2 L2A data for 68 of them and S2 16D data for 63. Using them together, we have data for 77 alerts.
- As this is still low data, we will
  - use pretrained encoder
  - create a cross validation dataset with 5 folds to get more robust performance on the full dataset
  - create one deforestation example per alert by using the first "before" image and the last "after" image with respect to the deforestation event. With good labels, we would like to change this by using "before" and "after images as close as possible to each other to train the model to detect deforestation events from one image to the next. But here, as our label timings are really not that good looking, I found that the before and after images right next to each other do not feature the deforestation changes.
  - create negative examples of deforestation by sampling inside the "before" images and adding an empty deforestation mask. Same for within the "after" images. This is important as in earlier experiments, I found the model to learn to predict always deforestation in the middle of the scene as this was a good baseline if every example features deforestation and is also roughly centered on it. Random cropping also helps here as it moves the deforestation position to the sides of the image.

## Create cross validation dataset

In [ ]:
df = gpd.read_file("data/mapbiomas_alerts.geojson")
events_sorted_by_area = df.sort_values("areaHa", ascending=False)["alertCode"].tolist()
df.shape

In [ ]:
paths_before_, paths_after_ = [], []
clouds_nodata_before_, clouds_nodata_after_ = [], []
label_paths, deforest_pxs = [], []
s2_valid_data_events = []
seed = 11219
seed_count = 0
for event_id in events_sorted_by_area[:]:
    s2_l2a_paths = sorted([k for k in glob.glob(f"data/{event_id}/sentinel2/*L2A*.tif") if not "ndvi" in k and not "ndwi" in k and not "compressed" in k])
    s2_16d_paths = sorted([k for k in glob.glob(f"data/{event_id}/sentinel2/*16D*.tif") if not "ndvi" in k and not "ndwi" in k and not "compressed" in k])
    s2_paths = s2_l2a_paths + s2_16d_paths
    if len(s2_paths) == 0:
        continue
    mask_path = glob.glob(f"data/{event_id}/*_mask.tif")[0]
    mask_date = mask_path.split("/")[-1].split('_')[1]
    mask_date_str = f"{mask_date[:4]}-{mask_date[4:6]}-{mask_date[6:8]}"
    mask_date = datetime.strptime(mask_date, "%Y%m%d")
    # print(mask_date)
    
    date = mask_path.split("/")[-1].split('_')[1]
    # Sort by the date right before the .tif
    paths = sorted(s2_paths, key=lambda p: datetime.strptime(p.split('_')[-1].split('.')[0], "%Y%m%d"))
    
    paths_after, paths_before = [], []
    clouds_nodata_after, clouds_nodata_before = [], []
    for path in paths:
        img_orig = tifffile.imread(path)
        if "L2A" in path:
            scl = img_orig[:, :, -2].copy()
            img_orig = img_orig[:, :, [11, 8, 4]]
        else:
            scl = img_orig[:, :, -1].copy()
            img_orig = img_orig[:, :, [10, 7, 3]]
        img_orig = img_orig.astype(np.float32)
        if scl.max() > 11:
            if scl.max() > 50:
                factor = 6
            elif scl.max() > 45:
                factor = 5
            elif scl.max() >= 40:
                factor = 4.5
            else:
                factor = 4
            img_orig = (img_orig / factor).round(0)
            scl_clean = (scl / factor).round(0).astype('uint8')   # back to 0..11
        else:
            factor = 1
            scl_clean = scl.copy()
        img_orig[img_orig == 0] = np.nan
        # low values, but all minimum above 1000 ==> Subtract by 1000
        if (np.nanmin(img_orig, axis=(0,1)) > 1000).sum() == 3:
            img_orig = img_orig - 1000
            img_orig[img_orig < 0] = np.nan
        
        cloudshadowmask = np.isin(scl_clean, [3, 8, 9, 10]).astype(np.uint8)   # clouds + shadows
        clouds_perc = 100 * cloudshadowmask.sum() / (img_orig.shape[0] * img_orig.shape[1])
        nodata_perc = 100 * np.isnan(img_orig[:, :, 0]).sum() / (img_orig.shape[0] * img_orig.shape[1])
        
        date = path.split("/")[-1].split('.')[0].split('_')[-1]
        date_str = f"{date[:4]}-{date[4:6]}-{date[6:8]}"
            
        date = datetime.strptime(path.split('_')[-1].split('.')[0], "%Y%m%d")
        if date > mask_date:
            paths_after.append(path)
            clouds_nodata_after.append(clouds_perc + nodata_perc)
        else:
            paths_before.append(path)
            clouds_nodata_before.append(clouds_perc + nodata_perc)
    print(f"{event_id}: {len(s2_l2a_paths):2} S2 L2A paths, {len(s2_16d_paths):2} 16D paths, {len(paths_before):2} before, {len(paths_after):2} after, ({mask_date})")
    if len(clouds_nodata_before) > 0 and len(clouds_nodata_after) > 0:
        print(f" --> min nodata/clouds before: {min(clouds_nodata_before):5.1f}%, after: {min(clouds_nodata_after):5.1f}%")
    if len(paths_before) > 0 and len(paths_after) > 0:
        label_path = glob.glob(f"data/{event_id}/*_mask.tif")[0]
        label_paths.append(label_path)
        label = tifffile.imread(label_path)
        deforest_pxs.append((label == 255).sum())
        
        s2_valid_data_events.append(event_id)
        paths_after_.append(paths_after)
        clouds_nodata_after_.append(clouds_nodata_after)
        paths_before_.append(paths_before)
        clouds_nodata_before_.append(clouds_nodata_before)

    if len(paths_before) > 2: # Create a new no deforestation example from the before images
        before_indices = np.arange(len(paths_before))
        np.random.seed(seed + seed_count)
        seed_count += 1
        np.random.shuffle(before_indices)
        sorted_indices_new_example = sorted(before_indices[:2])
        sorted_paths_new_example = [path for k, path in enumerate(paths_before) if k in sorted_indices_new_example]
        
        s2_valid_data_events.append(f"{event_id}_no_change_before")
        label_paths.append("empty")
        deforest_pxs.append(0)
        paths_after_.append([sorted_paths_new_example[1]])
        clouds_nodata_after_.append([clouds_nodata_before[sorted_indices_new_example[1]]])
        paths_before_.append([sorted_paths_new_example[0]])
        clouds_nodata_before_.append([clouds_nodata_before[sorted_indices_new_example[0]]])
        
    if len(paths_after) > 2: # Create a new no deforestation example from the before images
        after_indices = np.arange(len(paths_after))
        np.random.seed(seed + seed_count)
        seed_count += 1
        np.random.shuffle(after_indices)
        sorted_indices_new_example = sorted(after_indices[:2])
        sorted_paths_new_example = [path for k, path in enumerate(paths_after) if k in sorted_indices_new_example]
        
        s2_valid_data_events.append(f"{event_id}_no_change_after")
        label_paths.append("empty")
        deforest_pxs.append(0)
        paths_after_.append([sorted_paths_new_example[1]])
        clouds_nodata_after_.append([clouds_nodata_after[sorted_indices_new_example[1]]])
        paths_before_.append([sorted_paths_new_example[0]])
        clouds_nodata_before_.append([clouds_nodata_after[sorted_indices_new_example[0]]])
len(s2_valid_data_events)

In [ ]:
df = pd.DataFrame({"event_id": s2_valid_data_events, 
                   "deforest_pxs": deforest_pxs, "label_path": label_paths,
                   "paths_before": paths_before_, "clouds_nodata_before": clouds_nodata_before_,
                   "paths_after": paths_after_, "clouds_nodata_after": clouds_nodata_after_})
df.to_pickle("catalogues/2025_08_19_s2_rawdata.pkl")
df.shape

In [ ]:
df = pd.read_pickle("catalogues/2025_08_19_s2_rawdata.pkl")
df.shape

In [ ]:
def best_before_path(row):
    # FURTHEST AWAY FROM DEFORESTATON DATE
    clouds_nodata_before = row["clouds_nodata_before"]
    paths_before = row["paths_before"]
    
    # # CLOSEST TO DEFORESTATON DATE
    # clouds_nodata_before = row["clouds_nodata_before"][::-1]
    # paths_before = row["paths_before"][::-1]

    # Take the closest deforestation date with clouds < 3%
    for clouds_nodata, path in zip(clouds_nodata_before, paths_before):
        if clouds_nodata < 3:
            return path, clouds_nodata
    # Take the closest deforestation date with clouds < 10%
    for clouds_nodata, path in zip(clouds_nodata_before, paths_before):
        if clouds_nodata < 10:
            return path, clouds_nodata
    # Take the closest deforestation date with clouds < 30%
    for clouds_nodata, path in zip(clouds_nodata_before, paths_before):
        if clouds_nodata < 30:
            return path, clouds_nodata
    # Take the closest deforestation date with clouds < 100%
    for clouds_nodata, path in zip(clouds_nodata_before, paths_before):
        if clouds_nodata < 100:
            return path, clouds_nodata
    # Else return the closest path
    return paths_before[0], clouds_nodata_before[0]

def best_after_path(row):
    # FURTHEST AWAY FROM DEFORESTATON DATE
    clouds_nodata_after = row["clouds_nodata_after"][::-1]
    paths_after = row["paths_after"][::-1]
    
    # # CLOSEST TO DEFORESTATON DATE
    # clouds_nodata_after = row["clouds_nodata_after"]
    # paths_after = row["paths_after"]

    # Take the closest deforestation date with clouds < 3%
    for clouds_nodata, path in zip(clouds_nodata_after, paths_after):
        if clouds_nodata < 3:
            return path, clouds_nodata
    # Take the closest deforestation date with clouds < 10%
    for clouds_nodata, path in zip(clouds_nodata_after, paths_after):
        if clouds_nodata < 10:
            return path, clouds_nodata
    # Take the closest deforestation  date with clouds < 30%
    for clouds_nodata, path in zip(clouds_nodata_after, paths_after):
        if clouds_nodata < 30:
            return path, clouds_nodata
    # Take the closest deforestation date with clouds < 100%
    for clouds_nodata, path in zip(clouds_nodata_after, paths_after):
        if clouds_nodata < 100:
            return path, clouds_nodata
    # Else return the closest path
    return paths_after[0], clouds_nodata_after[0]
    
df["path_best_before"] = df[["paths_before", "clouds_nodata_before"]].apply(lambda row: best_before_path(row)[0], axis=1)
df["path_best_after"] = df[["paths_after", "clouds_nodata_after"]].apply(lambda row: best_after_path(row)[0], axis=1)

df["clouds_nodata_best_before"] = df[["paths_before", "clouds_nodata_before"]].apply(lambda row: best_before_path(row)[1], axis=1)
df["clouds_nodata_best_after"] = df[["paths_after", "clouds_nodata_after"]].apply(lambda row: best_after_path(row)[1], axis=1)

In [ ]:
df["clouds_nodata_best_before"].hist()

In [ ]:
df["clouds_nodata_best_after"].hist()

In [ ]:
# df = pd.read_pickle("catalogues/2025_08_19_s2_l2a_16d.pkl")
# df.shape

In [ ]:
print(df.shape)
df = df[(df["label_path"]=="empty") | ((df["clouds_nodata_best_before"] < 30) & (df["clouds_nodata_best_after"] < 30))]
print(df.shape)

In [ ]:
df.columns

In [ ]:
df["event_id"] = df["event_id"].apply(lambda x: str(x))

In [ ]:
df["event_id_for_folds"] = df["event_id"].apply(lambda x: x.split("_")[0])

In [ ]:
df["event_id_for_folds"].value_counts()

In [ ]:
def _assign_groups_to_folds(df, group_col, target_col, n_splits, seed):
    """Greedy randomized bin packing: assign whole groups to the fold with the smallest total."""
    tmp = df[[group_col, target_col]].copy()
    tmp[group_col] = tmp[group_col].astype(object).fillna("__MISSING_GROUP__")
    tmp[target_col] = pd.to_numeric(tmp[target_col], errors="coerce").fillna(0)

    grp_sum = tmp.groupby(group_col, dropna=False)[target_col].sum()
    groups = grp_sum.index.to_numpy()
    weights = grp_sum.to_numpy(dtype=float)

    if len(groups) < n_splits:
        raise ValueError(f"Need at least {n_splits} distinct groups in {group_col}.")

    rng = np.random.default_rng(seed)

    # Sort groups by descending weight with tiny random jitter to randomize ties.
    jitter = rng.random(len(weights)) * 1e-9
    order = np.argsort(-(weights + jitter))

    fold_sums = np.zeros(n_splits, dtype=float)
    assignment = {}

    # Ensure every fold gets at least one (the n_splits heaviest go one-per-fold).
    for i, idx in enumerate(order[:n_splits]):
        assignment[groups[idx]] = i
        fold_sums[i] += weights[idx]

    # Assign the rest to the currently lightest fold (tie-break randomly).
    for idx in order[n_splits:]:
        min_sum = fold_sums.min()
        candidates = np.flatnonzero(fold_sums == min_sum)
        fold = int(rng.choice(candidates))
        assignment[groups[idx]] = fold
        fold_sums[fold] += weights[idx]

    out = df.copy()
    out["_grp_"] = out[group_col].astype(object).fillna("__MISSING_GROUP__")
    out["fold"] = out["_grp_"].map(assignment).astype(int)
    out.drop(columns="_grp_", inplace=True)

    # Safety: each group in exactly one fold
    assert out.groupby(group_col)["fold"].nunique().max() == 1

    return out

def add_balanced_group_folds_until(
    df: pd.DataFrame,
    group_col: str = "event_id_for_folds",
    target_col: str = "deforest_pxs",
    n_splits: int = 5,
    std_threshold: float = 1250.0,
    start_seed: int = 0,
    max_seeds: int = 10_000,
    return_best_if_not_met: bool = True,
):
    """
    Try seeds start_seed .. start_seed+max_seeds-1.
    Return the first split with std < std_threshold.
    If none meets the threshold and return_best_if_not_met is True, return the best found.
    """
    best = {"std": float("inf"), "seed": None, "df": None}

    for seed in range(start_seed, start_seed + max_seeds):
        out = _assign_groups_to_folds(df, group_col, target_col, n_splits, seed)
        std_counts = out.groupby("fold")[target_col].sum().std()  # sample std (ddof=1), same as your snippet

        # Track best
        if std_counts < best["std"]:
            best = {"std": float(std_counts), "seed": seed, "df": out}

        if std_counts < std_threshold:
            out.attrs["fold_seed"] = seed
            out.attrs["fold_std"] = float(std_counts)
            return out  # success

    # No seed met the threshold
    if return_best_if_not_met and best["df"] is not None:
        out = best["df"]
        out.attrs["fold_seed"] = best["seed"]
        out.attrs["fold_std"] = best["std"]
        print(
            f"Warning: no seed achieved std<{std_threshold}. "
            f"Best std={best['std']:.3f} at seed={best['seed']}."
        )
        return out

    raise RuntimeError(
        f"No seed in range [{start_seed}, {start_seed+max_seeds-1}] achieved std<{std_threshold}."
    )

df = add_balanced_group_folds_until(
    df,
    group_col="event_id_for_folds",
    target_col="deforest_pxs",
    n_splits=5,
    std_threshold=1250,
    start_seed=0,
    max_seeds=10000,
)

In [ ]:
df.attrs["fold_std"]

In [ ]:
df.groupby("fold")["deforest_pxs"].sum()

To speed up data loading during training, it helps to save the tifs uncompressed.
- Even faster would be to do most preprocessing before, e.g. loading before/after images, eventual resizing, dropping of bands, normalizing, subtracting offsets, ... and in the dataloader only loading in the final array and doing augmentations.

In [ ]:
for idx, row in df.iterrows():
    output_path = row["path_best_before"].replace(".tif", "_uncompressed.tif")
    if not os.path.exists(output_path):
        with rasterio.open(row["path_best_before"]) as src:
            profile = src.profile
            profile["compress"] = None
            with rasterio.open(output_path, "w", **profile) as dst:
                dst.write(src.read())

    output_path = row["path_best_after"].replace(".tif", "_uncompressed.tif")
    if not os.path.exists(output_path):
        with rasterio.open(row["path_best_after"]) as src:
            profile = src.profile
            profile["compress"] = None
            with rasterio.open(output_path, "w", **profile) as dst:
                dst.write(src.read())

df["path_best_before"] = df["path_best_before"].apply(lambda x: x.replace(".tif", "_uncompressed.tif"))
df["path_best_after"] = df["path_best_after"].apply(lambda x: x.replace(".tif", "_uncompressed.tif"))

In [ ]:
df[["event_id", "deforest_pxs", "path_best_before", "path_best_after"]].sample(5)

In [ ]:
os.makedirs("catalogues", exist_ok=True)
df.to_pickle("catalogues/2025_08_19_s2_l2a_16d.pkl")
print(df.shape)

In [ ]:
df.groupby("fold")["deforest_pxs"].sum()

## Train

In [ ]:
exp_nb = "S2_l2a_16d_1-2"
# path = "catalogues/2025_08_19_s2_single.pkl"
path = "catalogues/2025_08_19_s2_l2a_16d.pkl"
hps_dict = {
    ############
    # Data
    ############
    "df_path": path,

    ############
    # Training
    ############

    ## Experiment Setup
    "name": f"Exp_{exp_nb}",

    ## Model
    "num_classes": 1,
    "input_channel": 13,
    "backbone": "timm_efficientnet_b1",
    "pretrained": "imagenet",
    "model": "unetplusplus",

    # Training Setup
    "print_freq": 500,
    "use_fp16": 0,
    "patience": 8,
    "epoch_start_scheduler": 10,

    # Optimizer
    "lr": 0.001,
    "weight_decay": 0.0,

    ## Data Augmentation on CPU
    "train_crop_size": 256 + 32 + 32,
    "train_batch_size": 16, #32
    "da_brightness_magnitude": 0.0,
    "da_contrast_magnitude": 0.0,

    # Data Augmentation on GPU
    "gpu_da_params": [0.25],
    
    ### Loss, Metric
    "alpha": 0.25,
           }

for fold_nb in range(0, 5):
    hps = HyperParams(**hps_dict)
    hps.fold_nb = fold_nb
    num_batches = 250 // (hps.train_batch_size * torch.cuda.device_count())
    # num_batches = 10
    hps.num_batches = num_batches
    train_dataset, train_loader, val_dataset, val_loader = get_dataloaders(hps)
    # continue

    best_metric, best_metric_epoch = train(hps, train_loader, val_loader)

In [ ]:
exp_nb = "S2_l2a_16d_1-1"
# path = "catalogues/2025_08_19_s2_single.pkl"
path = "catalogues/2025_08_19_s2_l2a_16d.pkl"
hps_dict = {
    ############
    # Data
    ############
    "df_path": path,

    ############
    # Training
    ############

    ## Experiment Setup
    "name": f"Exp_{exp_nb}",

    ## Model
    "num_classes": 1,
    "input_channel": 13,
    "backbone": "timm_efficientnet_b1",
    "pretrained": "imagenet",
    "model": "unetplusplus",

    # Training Setup
    "print_freq": 500,
    "use_fp16": 0,
    "patience": 8,
    "epoch_start_scheduler": 10,

    # Optimizer
    "lr": 0.001,
    "weight_decay": 0.0,

    ## Data Augmentation on CPU
    "train_crop_size": 256 + 32 + 32,
    "train_batch_size": 16, #32
    "cutmix_alpha": 0,
    "da_brightness_magnitude": 0.0,
    "da_contrast_magnitude": 0.0,

    # Data Augmentation on GPU
    "gpu_da_params": [0.25],
    
    ### Loss, Metric
    "alpha": 0.25,
           }

for fold_nb in range(0, 5):
    hps = HyperParams(**hps_dict)
    hps.fold_nb = fold_nb
    num_batches = 250 // (hps.train_batch_size * torch.cuda.device_count())
    hps.num_batches = num_batches
    train_dataset, train_loader, val_dataset, val_loader = get_dataloaders(hps)

    best_metric, best_metric_epoch = train(hps, train_loader, val_loader)

In [ ]:
# df[df["event_id"]==1387993]

In [ ]:
df[df["event_id"]==1388078]

In [ ]:
val_dataset.visualize(10)

In [ ]:
train_dataset.visualize(10)

In [ ]:
for k in range(100):
    print(np.unique(train_dataset[k]["mask"]))

In [ ]:
exp_nb = "S2_single_1-1"
path = "catalogues/2025_08_18_s2_single.pkl"
hps_dict = {
    ############
    # Data
    ############
    "df_path": path,

    ############
    # Training
    ############

    ## Experiment Setup
    "name": f"Exp{exp_nb}",

    ## Model
    "num_classes": 1,
    "input_channel": 15,
    "backbone": "timm_efficientnet_b1",
    "pretrained": 1,
    "model": "unetplusplus",

    # Training Setup
#     "resume": "trained_models/Exp7-4/fold_0/2022-02-05_23-13-25/best_metric_18_0.9768.pt",
    "print_freq": 500,
    "use_fp16": 0,
    "patience": 6, #5,
    "epoch_start_scheduler": 6,

    # Optimizer
    "lr": 0.001,
    "weight_decay": 0.0,

    ## Data Augmentation on CPU
    "train_crop_size": 400,
    "train_batch_size": 16, #32
    "cutmix_alpha": 0,
    "da_brightness_magnitude": 0.0,
    "da_contrast_magnitude": 0.0,

    # Data Augmentation on GPU
    "gpu_da_params": [0.25],
    
    ### Loss, Metric
    "alpha": 0.5,
#     "loss": "lovasz",
           }

for fold_nb in range(0, 5):
    hps = HyperParams(**hps_dict)
    hps.fold_nb = fold_nb
    num_batches = 250 // (hps.train_batch_size * torch.cuda.device_count())
    # num_batches = 10
    hps.num_batches = num_batches
    train_dataset, train_loader, val_dataset, val_loader = get_dataloaders(hps)
    # continue

    best_metric, best_metric_epoch = train(hps, train_loader, val_loader)

In [ ]:
import shutil
shutil.rmtree("trained_models/ExpS2_single_1-2/")

In [ ]:
exp_nb = "S2_single_1-2"
path = "catalogues/2025_08_18_s2_single.pkl"
hps_dict = {
    ############
    # Data
    ############
    "df_path": path,

    ############
    # Training
    ############

    ## Experiment Setup
    "name": f"Exp{exp_nb}",

    ## Model
    "num_classes": 1,
    "input_channel": 15,
    "backbone": "timm_efficientnet_b1",
    "pretrained": 1,
    "model": "unetplusplus",

    # Training Setup
#     "resume": "trained_models/Exp7-4/fold_0/2022-02-05_23-13-25/best_metric_18_0.9768.pt",
    "print_freq": 500,
    "use_fp16": 0,
    "patience": 6, #5,
    "epoch_start_scheduler": 10,

    # Optimizer
    "lr": 0.001,
    "weight_decay": 0.0,

    ## Data Augmentation on CPU
    "train_crop_size": 256 + 32 + 32,
    "train_batch_size": 16, #32
    "cutmix_alpha": 0,
    "da_brightness_magnitude": 0.0,
    "da_contrast_magnitude": 0.0,

    # Data Augmentation on GPU
    "gpu_da_params": [0.25],
    
    ### Loss, Metric
    "alpha": 0.5,
#     "loss": "lovasz",
           }

for fold_nb in range(0, 5):
    hps = HyperParams(**hps_dict)
    hps.fold_nb = fold_nb
    num_batches = 250 // (hps.train_batch_size * torch.cuda.device_count())
    # num_batches = 10
    hps.num_batches = num_batches
    train_dataset, train_loader, val_dataset, val_loader = get_dataloaders(hps)
    # continue

    best_metric, best_metric_epoch = train(hps, train_loader, val_loader)

## Create cross validation dataset with using the last after image

In [ ]:
df = gpd.read_file("data/mapbiomas_alerts.geojson")
events_sorted_by_area = df.sort_values("areaHa", ascending=False)["alertCode"].tolist()
df.shape

In [ ]:
paths_before_, paths_after_ = [], []
clouds_nodata_before_, clouds_nodata_after_ = [], []
s2_valid_data_events = []
for event_id in events_sorted_by_area[:]:
    s2_paths = sorted([k for k in glob.glob(f"data/{event_id}/sentinel2/*L2A*.tif") if not "ndvi" in k and not "ndwi" in k and not "uncompressed" in k])
    if len(s2_paths) == 0:
        continue
    mask_path = glob.glob(f"data/{event_id}/*_mask.tif")[0]
    mask_date = mask_path.split("/")[-1].split('_')[1]
    mask_date_str = f"{mask_date[:4]}-{mask_date[4:6]}-{mask_date[6:8]}"
    mask_date = datetime.strptime(mask_date, "%Y%m%d")
    # print(mask_date)
    
    date = mask_path.split("/")[-1].split('_')[1]
    # Sort by the date right before the .tif
    paths = sorted(s2_paths, key=lambda p: datetime.strptime(p.split('_')[-1].split('.')[0], "%Y%m%d"))
    # break
    paths_after, paths_before = [], []
    clouds_nodata_after, clouds_nodata_before = [], []
    for path in paths:
        # what_to_do_string = "SKIP"
        img_orig = tifffile.imread(path)
        # img_orig = img_orig.astype(np.float32)
        # img_orig = img_orig - 1000
        # img_orig[img_orig < 0] = 0
        clouds_perc = 100 * (img_orig[:, :, 2] > 3400).sum() / (img_orig.shape[0] * img_orig.shape[1])
        nodata_perc = 100 * (img_orig[:, :, 2] <= 1000).sum() / (img_orig.shape[0] * img_orig.shape[1])
        date = path.split("/")[-1].split('.')[0].split('_')[-1]
        date_str = f"{date[:4]}-{date[4:6]}-{date[6:8]}"
            
        date = datetime.strptime(path.split('_')[-1].split('.')[0], "%Y%m%d")
        if date > mask_date:
            paths_after.append(path)
            clouds_nodata_after.append(clouds_perc + nodata_perc)
        else:
            paths_before.append(path)
            clouds_nodata_before.append(clouds_perc + nodata_perc)
    print(f"{event_id}: {len(paths):2} S2 L2A paths, {len(paths_before):2} before, {len(paths_after):2} after, ({mask_date})")
    if len(clouds_nodata_before) > 0 and len(clouds_nodata_after) > 0:
        print(f" --> min nodata/clouds before: {min(clouds_nodata_before):5.1f}%, after: {min(clouds_nodata_after):5.1f}%")
    if len(paths_before) > 0 and len(paths_after) > 0:
        s2_valid_data_events.append(event_id)
        paths_after_.append(paths_after)
        clouds_nodata_after_.append(clouds_nodata_after)
        paths_before_.append(paths_before)
        clouds_nodata_before_.append(clouds_nodata_before)
len(s2_valid_data_events)

In [ ]:
df = pd.DataFrame({"event_id": s2_valid_data_events, 
              "paths_before": paths_before_, "clouds_nodata_before": clouds_nodata_before_,
                  "paths_after": paths_after_, "clouds_nodata_after": clouds_nodata_after_})
df.shape

In [ ]:
def best_before_path(row):
    # Take the earliest date with clouds < 10%
    for clouds_nodata, path in zip(row["clouds_nodata_before"], row["paths_before"]):
        if clouds_nodata < 10:
            return path, clouds_nodata
    # Take the earliest date with clouds < 30%
    for clouds_nodata, path in zip(row["clouds_nodata_before"], row["paths_before"]):
        if clouds_nodata < 30:
            return path, clouds_nodata
    # Take the earliest date with clouds < 100%
    for clouds_nodata, path in zip(row["clouds_nodata_before"], row["paths_before"]):
        if clouds_nodata < 100:
            return path, clouds_nodata
    # Else return the oldest path
    return row["paths_before"][0], row["clouds_nodata_before"][0]

def best_after_path(row):
    # Take the latest date with clouds < 10%
    for clouds_nodata, path in zip(row["clouds_nodata_after"][::-1], row["paths_after"][::-1]):
        if clouds_nodata < 10:
            return path, clouds_nodata
    # Take the latest date with clouds < 30%
    for clouds_nodata, path in zip(row["clouds_nodata_after"][::-1], row["paths_after"][::-1]):
        if clouds_nodata < 30:
            return path, clouds_nodata
    # Take the latest date with clouds < 100%
    for clouds_nodata, path in zip(row["clouds_nodata_after"][::-1], row["paths_after"][::-1]):
        if clouds_nodata < 100:
            return path, clouds_nodata
    # Else return the newest path
    return row["paths_after"][-1], row["clouds_nodata_after"][-1]
    
df["path_best_before"] = df[["paths_before", "clouds_nodata_before"]].apply(lambda row: best_before_path(row)[0], axis=1)
df["path_best_after"] = df[["paths_after", "clouds_nodata_after"]].apply(lambda row: best_after_path(row)[0], axis=1)

In [ ]:
df["clouds_nodata_best_before"] = df[["paths_before", "clouds_nodata_before"]].apply(lambda row: best_before_path(row)[1], axis=1)
df["clouds_nodata_best_after"] = df[["paths_after", "clouds_nodata_after"]].apply(lambda row: best_after_path(row)[1], axis=1)

In [ ]:
label_paths = []
deforest_pxs = []
for event_id in df["event_id"].tolist():
    label_path = glob.glob(f"data/{event_id}/*_mask.tif")[0]
    label_paths.append(label_path)
    label = tifffile.imread(label_path)
    deforest_pxs.append((label == 255).sum())
df["deforest_pxs"] = deforest_pxs
df["label_path"] = label_paths
df

In [ ]:
df.columns

In [ ]:
seed = 222221
while True:
    np.random.seed(seed)
    folds = [0, 1, 2, 3, 4] * (len(df) // 5 + 1) #np.random.choice([0, 1, 2, 3, 4], size=len(df))
    np.random.shuffle(folds)
    df["fold"] = folds[:len(df)]
    # print(df["fold"].value_counts())
    std_counts = df.groupby("fold")["deforest_pxs"].sum().std().item()
    # print(f'Seed {seed}: {df.groupby("fold")["deforest_pxs"].sum()}, Std = {std_counts:.0f}')
    if std_counts < 2000:
        break
    seed += 1

In [ ]:
for idx, row in df.iterrows():
    with rasterio.open(row["path_best_before"]) as src:
        output_path = row["path_best_before"].replace(".tif", "_uncompressed.tif")
        if not os.path.exists(output_path):
            profile = src.profile
            profile["compress"] = None
            with rasterio.open(output_path, "w", **profile) as dst:
                dst.write(src.read())

    with rasterio.open(row["path_best_after"]) as src:
        output_path = row["path_best_after"].replace(".tif", "_uncompressed.tif")
        if not os.path.exists(output_path):
            profile = src.profile
            profile["compress"] = None
            with rasterio.open(output_path, "w", **profile) as dst:
                dst.write(src.read())

df["path_best_before"] = df["path_best_before"].apply(lambda x: x.replace(".tif", "_uncompressed.tif"))
df["path_best_after"] = df["path_best_after"].apply(lambda x: x.replace(".tif", "_uncompressed.tif"))

In [ ]:
os.makedirs("catalogues", exist_ok=True)
df.to_pickle("catalogues/2025_08_18_s2_single_last_after_img.pkl")
print(df.shape)

In [ ]:
df.groupby("fold")["deforest_pxs"].sum()

## Train

In [ ]:
exp_nb = "S2_single_2-0"
path = "catalogues/2025_08_18_s2_single_last_after_img.pkl"
hps_dict = {
    ############
    # Data
    ############
    "df_path": path,

    ############
    # Training
    ############

    ## Experiment Setup
    "name": f"Exp{exp_nb}",

    ## Model
    "num_classes": 1,
    "input_channel": 15,
    "backbone": "timm_efficientnet_b1",
    "pretrained": 1,
    "model": "unetplusplus",

    # Training Setup
#     "resume": "trained_models/Exp7-4/fold_0/2022-02-05_23-13-25/best_metric_18_0.9768.pt",
    "print_freq": 500,
    "use_fp16": 0,
    "patience": 8, #5,
    "epoch_start_scheduler": 10,

    # Optimizer
    "lr": 0.001,
    "weight_decay": 0.0,

    ## Data Augmentation on CPU
    "train_crop_size": 256 + 32 + 32,
    "train_batch_size": 16, #32
    "cutmix_alpha": 0,
    "da_brightness_magnitude": 0.0,
    "da_contrast_magnitude": 0.0,

    # Data Augmentation on GPU
    "gpu_da_params": [0.25],
    
    ### Loss, Metric
    "alpha": 0.25,
#     "loss": "lovasz",
           }

for fold_nb in range(0, 5):
    hps = HyperParams(**hps_dict)
    hps.fold_nb = fold_nb
    num_batches = 250 // (hps.train_batch_size * torch.cuda.device_count())
    # num_batches = 10
    hps.num_batches = num_batches
    train_dataset, train_loader, val_dataset, val_loader = get_dataloaders(hps)
    # continue

    best_metric, best_metric_epoch = train(hps, train_loader, val_loader)